<a href="https://colab.research.google.com/github/KendallScott/ML2/blob/main/Scott_Kendall_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2

This is called DeathToGridSearch because with this example you will never have to think about how to manage a large number of classifiers etc simultaneously.  You will now be able to run and collect results in a very straightforward manner.  #LongLongLiveGridSearch!

In [74]:
import warnings
warnings.filterwarnings("ignore")

In [333]:
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder

from itertools import product
from sklearn.model_selection import StratifiedKFold, cross_val_predict, cross_val_score

import itertools
from itertools import *
from numpy import arange
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from xgboost import XGBClassifier

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset?resource=download

df = pd.read_csv('/content/drive/MyDrive/Employee.csv', encoding = 'ISO-8859-1',low_memory=False)

Mounted at /content/drive


In [5]:
df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [6]:
X = df.loc[:,df.columns != 'LeaveOrNot']
y = df['LeaveOrNot'].values.flatten()

In [7]:
df_num = X.select_dtypes(exclude='object')
num_cols = df_num.columns

# impute numeric with median
for i in num_cols:
    df_num[i] = df_num[i].transform(lambda x: x.fillna(pd.Series.mode(x)[0]))
df_num.info()

df_cat = X.select_dtypes(include='object')
cat_cols = df_cat.columns

# scale numerical data
scaler = StandardScaler()
df_num = scaler.fit_transform(df_num)

#one hot encode cat data
encoder = OneHotEncoder(sparse_output=False)
df_cat = encoder.fit_transform(df_cat)
X = np.hstack([df_num,df_cat])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4653 entries, 0 to 4652
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   JoiningYear                4653 non-null   int64
 1   PaymentTier                4653 non-null   int64
 2   Age                        4653 non-null   int64
 3   ExperienceInCurrentDomain  4653 non-null   int64
dtypes: int64(4)
memory usage: 145.5 KB


In [8]:
education = df[['Education', 'LeaveOrNot','City']].groupby(['Education', 'LeaveOrNot']).count().reset_index()
education

,Education,LeaveOrNot,City
0,Bachelors,0,2472
1,Bachelors,1,1129
2,Masters,0,447
3,Masters,1,426
4,PHD,0,134
5,PHD,1,45


### Dictionary functions

In [186]:
import re
#reference: https://stackoverflow.com/questions/69300562/how-to-define-the-grid-for-using-grid-search-from-scratch-in-python

#create the dictionary used to iterate and run grid search
def makeGrid(pars_dict):
    keys=pars_dict.keys()
    length = len(keys)
    combinations=itertools.product(*pars_dict.values())
    ds=[dict(zip(keys,cc)) for cc in combinations]
    return ds


In [350]:
def dict_cleanup(param_dict, new_dict):
  new_data = {}

  for  v in param_res.keys():
      new = re.sub('[^a-zA-Z_]', '', v)
      def slicedict(d, s):
        return {k:v for k,v in d.items() if k.startswith(new)}
      param_dict_copy = slicedict(param_res, new)
      new_data_values = list(param_dict_copy.values())
      new_data_empty = {}
      new_data_empty[new]=new_data_values

      new_data.update(new_data_empty)
  return new_data

### Grid Search Function

In [349]:
skf = StratifiedKFold(n_splits=3, shuffle=True)
skf.get_n_splits(X,y)

def search_results(model_type, model, grid, results, parameters):
  grid=grid
  model_type=model_type
  parameters = {}

  for x, i in enumerate(grid):
            model.set_params(**i)
            model.fit(X,y)
            score = cross_val_score(model, X, y, scoring='accuracy', cv=5)
            score=score.mean()
            results =np.append(results, np.ravel(score))
            if max(results)==score:
              #print('best model & params: ', i)
              best_model = i
              best_score = score

            #updating dictionary with results
            param_dict = {}
            param_dict.update(i)
            iteration = x
            param_dict.update(param_dict)
            param_dict["results"] = score
            param_dict["model"] = model_type
            param_dict["iteration"] = iteration

            keys=param_dict.keys()
            new_dict = {}

            for x, i in enumerate(keys):
                key =i + str(iteration)
                new_dict[key]=param_dict[i]

                parameters.update(new_dict)

                #new_dict[key] = param_dict[i]

  return parameters

### Running the custom grid search and dictionary functions

In [351]:
def grid_search(model_version):
  model_version = model_version
  model_selection = ['Logistic Regression', 'Random Forest', 'Linear SVC', 'XGBoost']
  #creating different params for different models in loop
  model_selection_results = {}
  grid_parameters = {}
  grid_results = np.empty(0)

  if model_version=='Logistic Regression':
    model=LogisticRegression(random_state=45,penalty='elasticnet',class_weight='balanced',solver='saga')
    l1 = arange(0, 1, 0.1)
    C = [  0.001,  0.1,  0.5, 1]
    max_iter = [25,  50, 60, 75, 100]

    hyper_dict = {'C': C, 'l1_ratio': l1, 'max_iter':max_iter}
    grid = makeGrid(hyper_dict)
    param = {}
    param_res =search_results(model_version, model, grid, grid_results, grid_parameters)

  if model_version=='Random Forest':
    model = RandomForestClassifier(criterion='gini',class_weight='balanced')
    n_estimators = [50, 100, 200]
    max_features = ['sqrt','log2',20]

    hyper_dict = {'n_estimators': n_estimators, 'max_features': max_features}
    grid = makeGrid(hyper_dict)
    param_res = {}
    param_res =search_results(model_version, model, grid, grid_results, grid_parameters)

  if model_version=='Linear SVC':
    model = LinearSVC(max_iter=100000,tol=0.01,dual=True,class_weight = 'balanced')
    C = [.001,.005,.01,.05,.1,.2,.5,.8,1,3,5,20,50]

    hyper_dict = {'C': C}
    grid = makeGrid(hyper_dict)
    param_res = {}
    param_res =search_results(model_version, model, grid, grid_results, grid_parameters)

  if model_version=='XGBoost':
    param_res = {}
    learning_rates = [0.1, 0.2]
    max_depths = [3, 5,7, 10, 20]
    gamma = [0,1,3]
    lambda_ls = [1,2,3]
    alpha = [0,0.1,1]

    hyper_dict = {'objective': 'binary:logistic', 'eval_metric': 'error', 'eta': learning_rates, 'max_depth': max_depths, 'gamma': gamma, 'lambda': lambda_ls, 'alpha': alpha}
    grid = makeGrid(hyper_dict)

    model = 'XGBClassifier'
    grid_results = np.empty(0)
    for x, i in enumerate(grid):

      for ids, (train_index, test_index) in enumerate(skf.split(X, y)):
        dtrain = xgb.DMatrix(X[train_index], label=y[train_index])
        dtest = xgb.DMatrix(X[test_index], label=y[test_index])
        evallist = [(dtrain, 'train'), (dtest, 'eval')]

        score = xgb.cv(params=i, num_boost_round=200, dtrain=dtrain,  nfold=3, stratified=True, early_stopping_rounds=3, verbose_eval=1)
        index=score.shape[0]-1
        score=1-(score.iloc[index,2])
        print('current score: ',score)
        grid_results = np.insert(grid_results,[0], score)

        #updating dictionary with results
        param_dict = {}
        param_dict.update(i)
        iteration = x
        param_dict.update(param_dict)
        param_dict["results"] = score
        param_dict["model"] = model
        param_dict["iteration"] = iteration

        keys=param_dict.keys()
        new_dict = {}

        for x, i in enumerate(keys):
          key =i + str(iteration)
          new_dict[key]=param_dict[i]
          param_res.update(new_dict)

  param_res_clean = {}
  param_res_clean = dict_cleanup(param_res, param_res_clean)

  return param_res_clean

In [ ]:
Logistic_Regression = grid_search('Logistic Regression')
Logistic_Regression

In [ ]:
Linear_SVC = grid_search('Linear SVC')
Linear_SVC

In [ ]:
Random_Forest = grid_search('Random Forest')
Random_Forest

In [ ]:
XG_Boost = grid_search('XGBoost')
XG_Boost